Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [1]:
!pip install pyspark

## Build Spark Session

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [3]:
train = spark.read.csv('/content/train.csv', inferSchema=True, header=True)
test = spark.read.csv('/content/test.csv', inferSchema=True, header=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [4]:
type(train)

pyspark.sql.dataframe.DataFrame

**Show 5 rows.**

In [5]:
train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [6]:
train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [7]:
train.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [8]:
train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [9]:
n_survived = train.groupBy('Survived').count()

**Display your result:**

In [10]:
n_survived.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [11]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

In [12]:
total = train.count()

In [13]:
def ratio(number):
    return number/total
    
get_ratio = udf(lambda x: ratio(x),FloatType())

In [14]:
n_survived.withColumn("ratio", ratio(col("count"))).show()

+--------+-----+------------------+
|Survived|count|             ratio|
+--------+-----+------------------+
|       1|  342|0.3838383838383838|
|       0|  549|0.6161616161616161|
+--------+-----+------------------+



**Can you get the number of males and females?**


In [15]:
gb_sex = train.groupBy('Sex').count()
gb_sex.show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



In [16]:
n_male = gb_sex.filter('Sex = "male"').select('count').collect()[0][0]
n_female = gb_sex.filter('Sex = "female"').select('count').collect()[0][0]

**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [17]:
train.groupBy(['Sex','Survived']).count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [18]:
def mratio(number):
    return number/n_male
    
get_ratio = udf(lambda x: mratio(x),FloatType())

In [19]:
train.filter('Sex = "male"').groupBy(['Survived']).count().withColumn("ratio", mratio(col("count"))).show()

+--------+-----+-------------------+
|Survived|count|              ratio|
+--------+-----+-------------------+
|       1|  109|0.18890814558058924|
|       0|  468| 0.8110918544194108|
+--------+-----+-------------------+



In [20]:
def fratio(number):
    return number/n_female
    
get_ratio = udf(lambda x: fratio(x),FloatType())

In [21]:
train.filter('Sex = "female"').groupBy(['Survived']).count().withColumn("ratio", fratio(col("count"))).show()

+--------+-----+-------------------+
|Survived|count|              ratio|
+--------+-----+-------------------+
|       1|  233| 0.7420382165605095|
|       0|   81|0.25796178343949044|
+--------+-----+-------------------+



**Create temporary view PySpark:**

In [22]:
train.createOrReplaceTempView('train')

**How many people survived, and how many didn't survive? By SQL:**

In [23]:
spark.sql('SELECT Survived,COUNT(Survived) AS Count FROM train GROUP BY Survived').show()

+--------+-----+
|Survived|Count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [24]:
spark.sql('SELECT Sex, SUM(Survived) / COUNT(Survived) AS Gender_ratio FROM train GROUP BY Sex').show() 

+------+-------------------+
|   Sex|       Gender_ratio|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [25]:
spark.sql('SELECT Pclass, SUM(Survived) / COUNT(Survived) AS Gender_ratio FROM train GROUP BY Pclass').show() 

+------+-------------------+
|Pclass|       Gender_ratio|
+------+-------------------+
|     1| 0.6296296296296297|
|     3|0.24236252545824846|
|     2|0.47282608695652173|
+------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [26]:
total=train.union(test)

**Display count:**

In [27]:
total.count()

1329

**Can you define the number of null values in each column?**


In [28]:
total.select([count(when(isnull(c), c)).alias(c) for c in total.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [29]:
total.select([count(when(isnull(c), c)).alias(c) for c in total.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Preprocessing 

**Create Temporary view PySpark:**

In [30]:
total.createOrReplaceTempView('total')

**Can you show the "name" column from your temporary table?**

In [31]:
spark.sql('SELECT name FROM total').show()

+--------------------+
|                name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [32]:
import pyspark.sql.functions as F
combined = total.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display "Title" column and count "Title" column:**

In [33]:
spark.sql('SELECT Title, count(Title) FROM combined group by Title').show()

+--------+------------+
|   Title|count(Title)|
+--------+------------+
|     Don|           1|
|    Miss|         257|
|Countess|           2|
|     Col|           4|
|     Rev|           9|
|    Lady|           2|
|  Master|          56|
|     Mme|           1|
|    Capt|           2|
|      Mr|         786|
|      Dr|          11|
|     Mrs|         186|
|     Sir|           2|
|Jonkheer|           2|
|    Mlle|           4|
|   Major|           3|
|      Ms|           1|
+--------+------------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [34]:
titles_map = {'Capt': 'rare',
 'Col': 'rare',
 'Countess': 'rare',
 'Don': 'rare',
 'Dr': 'rare',
 'Jonkheer': 'rare',
 'Lady': 'rare',
 'Major': 'rare',
 'Mlle': 'rare',
 'Mme': 'rare',
 'Ms': 'rare',
 'Rev': 'rare',
 'Sir': 'rare',
 'Miss': 'Miss',
 'Master': 'Master',
 'Mr': 'Mr',
 'Mrs': 'Mrs'}

**Run the function:**

In [35]:
def impute_title(title):
    return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [36]:
udf_data = udf(lambda x: impute_title(x),StringType())
title_df = combined.withColumn("Title",udf_data(combined['Title']))

**Display "Title" from table and group by "Title" column:**

In [37]:
title_df.groupBy("Title").count().show()

+------+-----+
| Title|count|
+------+-----+
|  rare|   44|
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
+------+-----+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [38]:
avg_age = combined.select(F.mean("Age")).collect()
avg_age[0][0]

30.079501879699244

**Fill missing with "age" mean:**

In [39]:
clean = combined.na.fill(avg_age[0][0],subset = ['Age'])
clean.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

## **Preprocessing Embarked**

In [41]:
clean.createOrReplaceTempView('clean')

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [42]:
grouped_Embarked = spark.sql('select Embarked, count(*) as Count from clean group by Embarked order by Count desc')


**Show "groupped_Embarked" your variable:**

In [43]:
grouped_Embarked.show()

+--------+-----+
|Embarked|Count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    3|
+--------+-----+



**Get max of groupped_Embarked:** 

In [44]:
mode = grouped_Embarked.first()[0]
mode

'S'

**Fill missing values with max 'S' of grouped_Embarked:**

In [45]:
clean = clean.na.fill(mode,subset = ['Embarked'])
clean.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [46]:
clean = clean.withColumn('Cabin', substring('Cabin', 1, 1))

**Show the result:**

In [47]:
clean.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|    C|       C|  Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|    C|       S|  Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|   Mr|
+-----------+--------+------+---

**Create the temporary view:**

In [48]:
clean.createOrReplaceTempView('clean')

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [49]:
clean.groupBy("Cabin").count().orderBy('count',ascending = False).show()

+-----+-----+
|Cabin|count|
+-----+-----+
| null| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**Fill missing values with "U":**

In [50]:
clean = clean.na.fill('U',subset = ['Cabin'])
final = clean.drop('Name','Ticket')
final.show()

+-----------+--------+------+------+------------------+-----+-----+-------+-----+--------+------+
|PassengerId|Survived|Pclass|   Sex|               Age|SibSp|Parch|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+------+------------------+-----+-----+-------+-----+--------+------+
|          1|       0|     3|  male|              22.0|    1|    0|   7.25|    U|       S|    Mr|
|          2|       1|     1|female|              38.0|    1|    0|71.2833|    C|       C|   Mrs|
|          3|       1|     3|female|              26.0|    0|    0|  7.925|    U|       S|  Miss|
|          4|       1|     1|female|              35.0|    1|    0|   53.1|    C|       S|   Mrs|
|          5|       0|     3|  male|              35.0|    0|    0|   8.05|    U|       S|    Mr|
|          6|       0|     3|  male|30.079501879699244|    0|    0| 8.4583|    U|       Q|    Mr|
|          7|       0|     1|  male|              54.0|    0|    0|51.8625|    E|       S|    Mr|
|          8|       

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

In [51]:
categoricalCols = ["Sex", "Embarked", "Title", "Cabin"]
indexOutputCols = [x + "_Index" for x in categoricalCols]


**StringIndexer(inputCol=None, outputCol=None)**

In [52]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')

**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [53]:
ohe_cols = [x + "_OHE" for x in categoricalCols]
ohe_encoder = OneHotEncoder(inputCols=indexOutputCols, outputCols=ohe_cols)

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [54]:
numericCols=['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [55]:
assemblerInputs = ohe_cols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [56]:
train_df, test_df = final.randomSplit([.8,.2],seed=42)
print(f"There are {train_df.count()} rows in the training set, and {test_df.count()} in the test set")

There are 1094 rows in the training set, and 235 in the test set


**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [57]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='Survived',featuresCol='features')
from pyspark.ml import Pipeline
pipeline =Pipeline(stages = [stringIndexer,ohe_encoder,vecAssembler,rf])

In [58]:
pipelineModel = pipeline.fit(train_df)

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [60]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

pred_df = pipelineModel.transform(test_df)

classificationEvaluator = MulticlassClassificationEvaluator(predictionCol='prediction',
                                         labelCol='Survived',
                                         metricName='accuracy')
accuracy = classificationEvaluator.evaluate(pred_df)
accuracy

0.8162393162393162

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
